<a href="https://colab.research.google.com/github/lyuguang/COMP90054/blob/main/solution_set_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP90054 AI Planning for Autonomy
### Problem Set 10
 - Policy approximation




### Key concepts:
- Policy approximation
- Policy gradients
- Actor critic



---


### Problem 1:


Discuss the following questions in groups:

- What is the difference between these different RL methods:
    - Policy approximation
    - Policy gradient
    - Actor critic

- What is the difference between these optimisation algorithms:
    - Gradient descent
    - Stochastic gradient descent
    - Approximate gradient descent


### Answer
1.
- Policy approximation is the general class of RL techniques in which the policy is a parameterised function directly, rather than being based on a value function.
- Policy gradient methods are a subset of policy approximation methods where the parameters of the policy are learned via gradient descent as opposed to other optimisation techniques (such as genetic algorithms)
- Actor-critic methods are a subset of policy approximation methods where a value function (the critic) is learned alongside the policy (the actor). This generally increases the efficiency compared to vanilla policy approximation.
2.
- Gradient descent is an optimisation method used to find the set of parameters which minimise some loss function. At each step, the exact gradient of the function is calculated, and parameters are updated in the direction of steepest descent.
- Since in RL, the loss function is often defined as an expected value, calculating the gradient precisely can be computationally prohibitive. Stochastic gradient descent instead simply samples the loss function to take a gradient - on average this will give the true gradient, but any given step might be pointing in a different direction.
- In RL, the loss function is generally the full return, requiring a Monte Carlo approach to even sample from it. If we use A bootstrapped TD target instead, then this is merely an approximation of the true loss function, so the gradients are also only approximations of the true gradients.

### Problem 2:

Recall the grid world from last lecture (details below). Now instead of doing linear-SARSA, Andrew the average computer scientist is trying to use a policy-gradient method. He has decided to use linear-softmax using the same features as last time. Recall from the lecture slides, a softmax policy is defined by: $\pi_\theta(s, a) \propto e^{f(s,a)^\top \theta}$
1. Assume weights of [0.1, -0.2]. What is the probability of taking each action from the state (2,3)?
2. Does using a policy gradient method improve upon the issue of having a goal in the middle of the board? Does it solve it?
3. After moving up from (2,3), the agent reaches the goal 25 timesteps later. Calculate the change in weights from this transition assuming the REINFORCE algorithm (MC policy gradient) is used with $\alpha = 0.2$.
4. (Bonus) Is there a single set of parameters that give the optimal policy?
5. (Bonus) Recalculate the weight update from question 3 using Advantage-Actor Critic instead of REINFORCE, assume the value function $V$ uses $(x,y)$ as features (not $x',y'$ as it is independent of the action) and currently has weights of [0.05, 0.03].



Environment recap:
Consider a simple MDP consisting of a 10x10 grid, the agent can move up, down, left, or right. The agent starts in (0,0) and gets a reward of +1 when reaching (9,9) which is a terminal state. $\gamma = 0.9$, and there is no randomness. Andrew the average computer scientist has designed the following feature representation: $f((x,y),A) =  (x',y')$, where $x'$ and $y'$ are the coordinates of the agent after applying action $A$.


### Answer

Note - the answers to Q3 and Q4 may be different to what is discussed in class. An issue was identified with the gradient term, see if you can spot the difference!

1. We must calculate $f((2,3),a)^\top \theta$ for all actions. This gives:
   
  - Up:       $(2\times 0.1+4\times -0.2) = -0.6$
  - Left:     $(1\times 0.1+3\times -0.2) = -0.5$
  - Right:    $(3\times 0.1+3\times -0.2) = -0.3$
  - Down:     $(2\times 0.1+2\times -0.2) = -0.2$
   
Now the un-normalised likelihoods are:
   - Up:       $e^{-0.6} = 0.54881$
   - Left:     $e^{-0.5} = 0.60653$
   - Right:    $e^{-0.3} = 0.74082$
   - Down:     $e^{-0.2} = 0.81873$
   
Normalising to probabilities, gives:
   - Up:       $0.54881/2.71489 = 0.202$
   - Left:     $0.60653/2.71489 = 0.223$
   - Right:    $0.74082/2.71489 = 0.273$
   - Down:     $0.81873/2.71489 = 0.302$

2. It improves the problem, because now we can reasonably learn a probabilistic policy, so the agent can act randomly. This is better than a determinstic one, since there is a chance of reaching the reward from any square, but is obviously not optimal. Without some non-linearity or better features, the problem still persists.
3. Recall from the notes, that the update step is given by: $\Delta \theta_t = \alpha \nabla_\theta \log \pi_\theta(s_t, a_t) G_t$. For linear softmax,
   $$\nabla_\theta \log \pi_\theta(s_t, a_t) = \nabla_\theta \log e^{\frac{f(s,a)^\top \theta}{\sum_{a'} f(s,a')^\top \theta}}$$
   $$ = f(s,a) - \sum_{a'} \pi(s,a')\cdot f(s,a')$$
   $G_t$, the return, is $\gamma^{25}\times 1 = 0.07179$.
   The gradient term can be calculated using the probabilities above:
   $$f(s,a) - \sum_{a'} \pi(s,a')\cdot f(s,a')$$
   $$[2,4] - (0.202*[2,4] +0.223*[1,3] + 0.273*[3,3]+  0.302*[2,2])$$
   $$[-0.05,-0.9]$$
   So, $\Delta \theta_t = \alpha\times f(s,a)\times \gamma^{25} = 0.2\times [-0.05,-0.9]\times 0.07179$, which means
   $\Delta \theta_0 = -0.0007$ and $\Delta \theta_1 = 0.-0.0129$. The new weights are therefore: $[0.0993, -0.2129]$.
4. No, since the probability of taking an action is determined by the *relative* weights of each action, scaling everything up or down can leave the actual policy unchanged. Compare this to a Q-learning approach where the estimates are anchored to the actual estimates of future discounted reward.
5. The update step now uses the TD error: $\delta_{v} = r + \gamma V_{v}(s') - V_{v}(s)$ as an estimate of the advantage instead of the full return $G_t$.
   According to the parameterised value function, $V(s) = (2\times 0.05+3\times 0.03) = 0.19$ and $V(s') = (2\times 0.05+4\times 0.03) = 0.22$.
   
   So, $\delta_{v} = 0 + 0.9\times0.22 - 0.19 = 0.008$, which means $\Delta \theta_t = 0.2\times [2, 4]\times 0.08$, and the new weights are $[0.1032, -0.1936]$.


---

### Problem 3 (optional - not covered in class):



Consider the following questions:
- Why are value-based RL methods inappropriate for continuous action spaces? How could you apply them anyway? How do policy-based methods address this challenge instead?
- What are the two major changes PPO makes from A2C? How do these improve efficiency?
- What is the purpose of the entropy term in the loss function in PPO or other policy gradient methods?


### Answer

- In order to use a greedy (or $\epsilon$-greedy) policy defined over a value function, we must able to compute the maximum over all the possible actions. In a continuous action space, this may be computationally prohibitive to do, as it is itself an entire optimisation problem. Instead, we can simple discretise the action space into a discrete one, but this suffers from two main downsides. Firstly, you lose the ability to perform very precise control. Secondly, due to the 'curse of dimensionality', even a relatively coarse discretisation will blow up to an infeasible number of actions if the dimensions of the action space is large. For example in a robotic setting with 27 motors controlling different joints, even discretising to only 5 possible values would result in $5^{27} = 7.5e18$ actions.
Instead, policy based methods directly produce an action without having to calculate the maximum. In a continuous setting, this is often done by having $f(s,a)^\top \theta$ give the mean of a normal distribution, which is then sampled to return the action.

- The first key change is clipping the gradient so that the weights do not change too far in a single step. This helps to mitigate bias due to the Advantage function - if the policy suddenly changes a lot, then the value-estimates trained on the old policy may be quite unreliable, and this introduces a bias into the update steps. Second, experience is collected and then repeatedly trained over in mini-batches (like batch methods in value-approximation RL). This is only possible due to the clipped gradient as otherwise repeated training tends to be unstable. Note that these are largely empirical results - there is minimal theoretical basis for these choices, they just turn out to work well in practice.

- Having entropy included in the loss function is a way of encouraging exploration. Unlike in value-based methods, policy-approximation methods can not just use an epsilon-greedy policy to encourage exploration. Instead, having an entropy term discourages the policy from converging to a an overly concentrated distribution, ensuring that a variety of actions are still chosen.